In [3]:
print("Notebook is working")

Notebook is working


In [8]:
import requests

#Weather API endpoint
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": date,
    "end_date": date, 
    "hourly": "winddirection_10m, windspeed_10m",
    "timezone": "America/New_York"
}

In [5]:
#Citizen's Bank Park (Phillies Stadium)
latitude = 39.9057
longitude = -75.1665

date = '2025-06-30'